In [ ]:
import numpy as np
from my_functions import *
import matplotlib.pyplot as plt
import pandas as pd
import glob

In [ ]:
split_converter = lambda s: s.split()
sum_flags = lambda s: np.sum(np.array(s.split()).astype(float))
tile_name = lambda s: s[:8]

cat = pd.read_csv('csv/minijpas.MagAB_aper3_photoz_gaia_4.csv', sep=',', header=1,
    converters={2: split_converter, 3: split_converter, 4: sum_flags,
    5: sum_flags, 21: tile_name})

cat = cat[np.array([len(x) for x in cat['MAG_APER_3_0']]) != 0]
cat = cat[np.array(cat['FLAGS']) + np.array(cat['MASK_FLAGS']) == 0] 

In [ ]:
w_central = central_wavelength().reshape(-1, 1)

In [ ]:
# # Load QSO catalog
# filename = ('/home/alberto/cosmos/JPAS_mocks_sep2021/'
#     'JPAS_mocks_classification_19nov_model11/Magnitudes_model_11/Qso_jpas_mock_mag_train.cat')

# my_filter_order = np.arange(60)
# my_filter_order[[-4, -3, -2, -1]] = np.array([1, 12, 28, 43])
# my_filter_order[1:-4] += 1
# my_filter_order[12:-4] += 1
# my_filter_order[28:-4] += 1
# my_filter_order[43:-4] += 1

# qso_flx = pd.read_csv(
#     filename, sep=' ', usecols=range(2, 2 + 60)
# ).to_numpy().T[my_filter_order]
# qso_err = pd.read_csv(
#     filename, sep=' ', usecols=range(2 + 60, 2 + 60 + 60)
# ).to_numpy().T[my_filter_order]
# qso_zspec = pd.read_csv(filename, sep=' ', usecols=[127]).to_numpy().reshape(-1, )

# # Randomly sample sources corresponding to 200 deg2
# idx = np.arange(100_000)

In [ ]:
# Load my QSO catalog

filename = '/home/alberto/almacen/Source_cats/QSO_100000_v4/data.csv'

data = pd.read_csv(filename)
qso_flx = data.to_numpy()[:, 1 : 60 + 1].T
qso_err = np.abs(data.to_numpy()[:, 60 + 1 : 120 + 1].T)

qso_err = -flux_to_mag(qso_flx + qso_err, w_central) + flux_to_mag(qso_flx, w_central)
qso_flx = flux_to_mag(qso_flx, w_central)
nanmag = np.isnan(qso_flx)
qso_flx[nanmag] = 99.
qso_err[nanmag] = 99.


In [ ]:
## Load SF catalog

filename = '/home/alberto/almacen/Source_cats/LAE_10deg_z2-4_v4/'
files = glob.glob(filename +'data*')
files.sort()
fi = []

for name in files:
    fi.append(pd.read_csv(name))

data = pd.concat(fi, axis=0, ignore_index=True)

sf_flx = data.to_numpy()[:, 1 : 60 + 1].T
sf_err = np.abs(data.to_numpy()[:, 60 + 1 : 120 + 1].T)

sf_err = -flux_to_mag(sf_flx + sf_err, w_central) + flux_to_mag(sf_flx, w_central)
sf_flx = flux_to_mag(sf_flx, w_central)
nanmag = np.isnan(sf_flx)
sf_flx[nanmag] = 99.
sf_err[nanmag] = 99.

In [ ]:
params_tiles = []
for tile in ['AEGIS002']:
    flx = np.stack(
        cat['MAG_APER_3_0'].to_numpy()[np.where(cat['name'] == tile)[0]]
    ).astype(float).T
    flx_err = np.stack(
        cat['MAG_ERR_APER_3_0'].to_numpy()[np.where(cat['name'] == tile)[0]]
    ).astype(float).T

In [ ]:
detec_lim = np.vstack(
    (
        pd.read_csv('csv/5sigma_depths_NB.csv', header=None),
        pd.read_csv('csv/5sigma_depths_BB.csv', header=None)
    )
)[:, 1]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

nb_c = 11
mask = (flx[nb_c] < 25)
qso_mask = (qso_flx[nb_c] < 25)
sf_mask = (sf_flx[nb_c] < 25)
ax.scatter(flx[nb_c, mask], flx_err[nb_c, mask], label='miniJPAS', zorder=99, alpha=0.2)
ax.scatter(qso_flx[nb_c, qso_mask], qso_err[nb_c, qso_mask], label='QSO mock', alpha=0.2)
ax.scatter(sf_flx[nb_c, sf_mask], sf_err[nb_c, sf_mask], label='SF mock')

# x_Arr = np.linspace(1e-20, 1e-16, 100)

ax.axvline(detec_lim[nb_c], ls='--', color='k')

ax.set_xlim((18, 24))
ax.set_ylim((0, 1))

ax.legend(fontsize=15)

ax.set_xlabel('r (magAB)', fontsize=15)
ax.set_ylabel('$\sigma$(r) (magAB)', fontsize=15)
ax.set_title(f'NB {nb_c}', fontsize=20)

# ax.set_yscale('log')
# ax.set_xscale('log')

plt.show()